In [1]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
from PIL import Image
import requests
import json
from tqdm import tqdm

/home/gpuuser3/miniconda3/envs/mmsd_annotation/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")
model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", torch_dtype=torch.float16, low_cpu_mem_usage=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  5.83it/s]


LlavaNextForConditionalGeneration(
  (vision_tower): CLIPVisionModel(
    (vision_model): CLIPVisionTransformer(
      (embeddings): CLIPVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(577, 1024)
      )
      (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-23): 24 x CLIPEncoderLayer(
            (self_attn): CLIPSdpaAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (mlp): CLIPMLP(
              (activation_fn

In [3]:
!nvidia-smi

Fri Dec 27 17:03:59 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 12.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:4E:00.0 Off |                    0 |
| N/A   28C    P0   127W / 400W |  59141MiB / 81251MiB |     32%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:

def save_json(file_name, data):
    with open(file_name, 'w', encoding='utf-8') as f:
        f.write(json.dumps(data, indent=4))


def load_json(file_path):
    with open(file_path, "r") as f:
        return json.load(f)


def process_response(response):
    answer = response.split('```json')[-1].split('```')[0]
    try:
        resp_as_dict = json.loads(answer)
        reason  = ''
        label = ''

        label = resp_as_dict.get('Label', '-1')
        reason = resp_as_dict.get('Reason', 'Not Applicable')
    except Exception as e:
        print(answer)
        label = '-1'
        reason = 'Not Applicable'


    return reason, label

In [17]:
def image_prompt_2() -> list[dict]:
    '''4 Shot CoT Prompt'''
    prompt = """
    Given an image, analyze its visual elements and text (if any) to determine if it contains sarcasm, then classify it as either "Sarcastic" or "Not Sarcastic". 
    
    The definitions are:
    "Sarcastic": The image contains elements that deliberately contrast with or contradict each other to create irony or mockery. This includes: exaggerated reactions, obvious mismatches between text and image, deliberately incorrect labels, or situations that highlight absurdity.
    "Not Sarcastic": The image presents its content straightforwardly without ironic intent. The visual and textual elements align naturally and don't create deliberate contradictions.

    **Output Format:**
    {
        "Thinking": "<Your thinking>",
        "Conclusion": "Sarcastic / Not Sarcastic",
        "Reasoning": "<Your Reasoning>"
    }

    **Note**
    - Adhere **strictly** to this output format.
    """

    messages = [
            {"role": "system", "content": [{"type": "text", "text": prompt}]},
            {"role": "user", "content": [{"type": "text", "text": "Is this image sarcastic?"}, {"type": "image"}]},
            {"role": "assistant", "content": [{
                "type": "text", # 1dhp7l8.png
                "text": """
                        {
                            "Thinking": "The image shows a woman next to a bear statue with accompanying text suggesting an absurd scenario about lifting an Xbox onto a roof. The content employs exaggerated expressions, visual contradictions, and deadpan contrasts. While presenting a straightforward scenario, it's clearly hyperbolic and not meant to be taken literally. The cultural context suggests playful relationship dynamics, and the creator's intent appears to be a humorous exaggeration of physical dominance.",
                            "Conclusion": "Sarcastic",
                            "Reasoning": "The humorous text combined with the visual absurdity suggests a playful and exaggerated premise, indicative of sarcasm."
                        }
                """
            }]},
            {"role": "user", "content": [{"type": "text", "text": "Is this image sarcastic?"}, {"type": "image"}]},
            {"role": "assistant", "content": [{ 
                "type": "text", # l23rnt.png
                "text": """
                        {
                            "Thinking": "The image features a cartoonish hot dog character dancing on a hot dog roller grill, accompanied by a caption about dancing on ashes. The hot dog displays exaggerated, cheerful expressions, creating a visual contradiction where an animated food item appears in a setting related to its demise. The absurd juxtaposition of a dancing hot dog among other hot dogs, combined with a deadpan contrast between the playful character and morbid caption, suggests sarcasm. While the image directly shows the hot dog and grill, the situation of dancing food items is atypical. The image makes a humorous reference to hot dog serving practices, with the creator's intent appearing to be humor through a visual gag. The caption adds a dark twist, and the overall synthesis shows sarcasm through the contrast between the cheerful image and serious caption.",
                            "Conclusion": "Sarcastic",
                            "Reasoning": "The combination of the playful hot dog character and the serious tone of the caption suggests a sarcastic interpretation, as it juxtaposes a light-hearted image with a morbid theme."
                        }
                """
            }]},
            {"role": "user", "content": [{"type": "text", "text": "Is this image sarcastic?"}, {"type": "image"}]},
            {"role": "assistant", "content": [{
                "type": "text", # 1d60qm8.png
                "text": """
                        {
                            "Thinking": "The image shows a person working on a crochet project with a cat resting on their lap. The subject matter is crochet work and a relaxed white cat, with no accompanying text. The scene lacks any sarcastic elements: there are no exaggerated expressions, visual contradictions, absurd juxtapositions, or deadpan contrasts. The image directly represents a common leisure activity and presents a literal scene without hidden meanings, showing a typical interaction between a person and their pet while engaging in a craft. There are no cultural references suggesting sarcasm. The purpose appears to be showcasing a peaceful moment with a pet while crocheting. The crochet hook suggests a relaxing activity but doesn't indicate sarcasm. Overall, there is minimal to no evidence of sarcasm present in the scene, and there is little ambiguity as the scene appears straightforward.",
                            "Conclusion": "Not Sarcastic",
                            "Reasoning": "The image depicts a straightforward and cozy interaction between a person and their cat during a crafting activity. There are no indicators of sarcasm such as exaggerated expressions or absurdity present."
                        }
                """
            }]},
            {"role": "user", "content": [{"type": "text", "text": "Is this image sarcastic?"}, {"type": "image"}]},
            {"role": "assistant", "content": [{
                "type": "text", # ta5wmi.png
                "text": """
                        {
                            "Thinking": "The image displays a standard gas station with pricing signage, showing fuel prices in a straightforward manner. The composition emphasizes fuel availability and clear price display. There are no signs of sarcasm: no exaggerated expressions, visual contradictions, absurd juxtapositions, or deadpan contrasts were detected. The representation is direct and literal, showing typical gas station pricing in a common setting. No cultural references or additional elements like emojis suggest any alternate interpretation. The purpose appears purely informational, focusing on communicating fuel prices to customers. The presentation maintains low ambiguity with clear communication intent.",
                            "Conclusion": "Not Sarcastic",
                            "Reasoning": "The image demonstrates a straightforward display of fuel prices without any elements suggesting sarcasm, exaggeration, contradiction, or irony. The focus is entirely on clear, factual communication of pricing information."
                        }
                """
            }]},

        ]
    return messages

In [18]:

import os

data_path = './data/samples/'
image_file_path = './data/images/'
new_data_path = './data/samples/'
file = "train"


def sendMessageText(instruction, text=None):
    # Create the conversation based on initial history and new message
    conversation = [
         {
            "role": "user",
            "content": [
                {"type": "text", "text": text},
            ],
        },
    ]
        
    conversation = instruction + conversation

    # print(conversation)

    # Generate the prompt
    prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

    # Prepare the inputs
    inputs = processor(images=None, text = prompt, return_tensors="pt").to(device)

    # Autoregressively complete the prompt
    try:
        output = model.generate(**inputs, max_new_tokens=1024, pad_token_id = 2)
    except Exception as e:
        print(e)
        return ''
    response_text = processor.decode(output[0], skip_special_tokens=True)

    return response_text

def sendMessageImage(instruction, image=None):
    # Create the conversation based on initial history and new message
    conversation = [
         {
            "role": "user",
            "content": [
                {"type": "text", "text": "Is this image sarcastic?"},
                {"type": "image"},
            ],
        },
    ]
        
    conversation = instruction + conversation

    # print(conversation)

    # Generate the prompt
    prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
    img_ids = ["1dhp7l8", "l23rnt", "1d60qm8", "ta5wmi"]
    images = []
    for id in img_ids:
        images.append(Image.open(f"{image_file_path}/{id}.png"))
        
    images.append(image)
    # Prepare the inputs
    inputs = processor(images=images, text=prompt, padding=True, return_tensors="pt").to(model.device)

    # Autoregressively complete the prompt
    try:
        generate_ids = model.generate(**inputs, max_new_tokens=1024)
    except Exception as e:
        print(e)
        return ''
    response_text = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)

    return response_text

In [19]:
print(processor.apply_chat_template(image_prompt_2(), add_generation_prompt=True))

<<SYS>>

    Given an image, analyze its visual elements and text (if any) to determine if it contains sarcasm, then classify it as either "Sarcastic" or "Not Sarcastic". 
    
    The definitions are:
    "Sarcastic": The image contains elements that deliberately contrast with or contradict each other to create irony or mockery. This includes: exaggerated reactions, obvious mismatches between text and image, deliberately incorrect labels, or situations that highlight absurdity.
    "Not Sarcastic": The image presents its content straightforwardly without ironic intent. The visual and textual elements align naturally and don't create deliberate contradictions.

    **Output Format:**
    {
        "Thinking": "<Your thinking>",
        "Conclusion": "Sarcastic / Not Sarcastic",
        "Reasoning": "<Your Reasoning>"
    }

    **Note**
    - Adhere **strictly** to this output format.
    
<</SYS>>

[INST] <image>
Is this image sarcastic? [/INST] 
                        {
            

In [22]:
resp = sendMessageImage(image_prompt_2(), Image.open('data/images/1aco83f.png'))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [23]:
print(resp[0])

<<SYS>>

    Given an image, analyze its visual elements and text (if any) to determine if it contains sarcasm, then classify it as either "Sarcastic" or "Not Sarcastic". 
    
    The definitions are:
    "Sarcastic": The image contains elements that deliberately contrast with or contradict each other to create irony or mockery. This includes: exaggerated reactions, obvious mismatches between text and image, deliberately incorrect labels, or situations that highlight absurdity.
    "Not Sarcastic": The image presents its content straightforwardly without ironic intent. The visual and textual elements align naturally and don't create deliberate contradictions.

    **Output Format:**
    {
        "Thinking": "<Your thinking>",
        "Conclusion": "Sarcastic / Not Sarcastic",
        "Reasoning": "<Your Reasoning>"
    }

    **Note**
    - Adhere **strictly** to this output format.
    
<</SYS>>

[INST]  
Is this image sarcastic? [/INST] 
                        {
                  

In [33]:
process_response(resp)

('The image does not show any elements of sarcasm.', '0')

In [38]:
from transformers import logging
logging.set_verbosity_warning()

In [ ]:
# Check first if there are any checkpoints present
if os.path.exists(f'{data_path}/{file}.json'):
    #datas = load_json(f'{new_data_path}/new_{file}.json')
    datas = load_json(f'{data_path}/{file}.json')
    # print(f"datas : {datas}")
else:
    print(f'File paths not found')
    
count = 0
for line in datas:
    #print(line)
    if 'llava_mix_info' in line:
        continue
    
    img_name = f"{line['submissionId']}.png"
    text = line['utterance']
    if not os.path.exists(f'{image_file_path}/{img_name}'):
        continue
    
    img = f'{image_file_path}/{img_name}'
    image = Image.open(img)

    instruction = image_prompt()
    response1 = sendMessage(instruction, None, image)
    # print(f'Response1:\n {response1}')
    reason1, label1 = process_response(response1)
    if reason1 == '' or label1 == '':
        line['llava_img_response'] = response1
    line['llava_img_info'] = reason1
    line['llava_img_label'] = label1

    instruction = text_prompt()
    response2 = sendMessage(instruction, text, None)
    # print(f'Response2:\n {response2}')
    reason2, label2 = process_response(response2)
    if reason2 == '' or label2 == '':
        line['llava_text_response'] = response2
    line['llava_text_info'] = reason2
    line['llava_text_label'] = label2

    instruction = multimodal_prompt()
    response3 = sendMessageMultimodal(instruction, text, image)
    # print(f'Response3 \n{response3}')
    reason3, label3 = process_response(response3)
    if reason3 == '' or label3 == '':
        line['llava_mix_response'] = response3
    line['llava_mix_info'] = reason3
    line['llava_mix_label'] = label3


    print('*****************************')
    print(f"Image name: {img_name}")
    print(f"Text: {text}")
    print(f"Reason1 {reason1}")
    print(f"Label1 {label1}")
    print(f"Reason2 {reason2}")
    print(f"Label2 {label2}")
    print(f"Reason3 {reason3}")
    print(f"Label3 {label3}")
    print('*****************************')

    count += 1
    if count == 50:
        # This is a checkpoint to prevent unexpected stops during code execution and loss of previous results.
        print('save_a_part')
        count = 0
        save_json(f'{new_data_path}/new_{file}.json', datas)

save_json(f'{new_data_path}/new_{file}.json', datas)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



{
    "Thinking": "The image shows a cat sitting on a bed with a blanket. The cat appears to be looking at the camera, and its position on the bed suggests it might be seeking attention or comfort. The supporting text contains a question and a response, which seems to be a conversation or an exchange of ideas. The question 'are you a zoophile?' is a term used to describe someone who is sexually attracted to animals. The response 'me: [/] 1
}

*****************************
Image name: btzlgz.png
Text: -are you a zoophile? -me:
Reason1 The image does not contain any text or caption, and the cat's expression and position do not provide any clear indication of sarcasm.
Label1 0
Reason2 The text is a simple question and does not contain any contrasts, contradictions, or ironic elements.
Label2 0
Reason3 Not Applicable
Label3 -1
*****************************


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


*****************************
Image name: j4r0go.png
Text: “mom? halp”
Reason1 The image depicts a contrast between the calm demeanor of the cat and the alertness of the rabbit, which creates a humorous or ironic situation. The rabbit's position in front of the carrier, looking directly at the camera, adds to the sarcastic tone of the image.
Label1 1
Reason2 The text is a non-sarcastic request for help.
Label2 0
Reason3 The contrast between the rabbit's caption and the cat's apparent comfort in the cage creates an ironic and sarcastic effect.
Label3 1
*****************************


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


*****************************
Image name: 1945rc.png
Text: dude, get a room.
Reason1 The image does not contain any text or visual elements that suggest sarcasm.
Label1 0
Reason2 The text contains a sarcastic element.
Label2 1
Reason3 The caption's sarcastic tone contrasts with the cat's relaxed posture, creating a humorous and sarcastic effect.
Label3 1
*****************************


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



*****************************
Image name: i2ofd6.png
Text: elijah, you weigh 12 pounds and earn approximately negative 100 dollars a month. why are you taking up 75% of the desk chair?
Reason1 The image does not contain any sarcastic elements.
Label1 0
Reason2 The text contains sarcastic elements and is likely intended to be humorous.
Label2 1
Reason3 Not Applicable
Label3 -1
*****************************


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
